In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as rmse
from sklearn.linear_model import LinearRegression

In [49]:
df = pd.read_csv(r"C:\Users\TheDarkAce\Jupyter scripts\Datathon Chevron\filesForStartOfDatathon/preprocessed.csv")

# removing some outliers from target variable
df = df[df["rate_of_penetration"] < 270]

In [50]:
df.head()

,segment_id,wellbore_chev_no_id,area_id,formation_id,bit_model_id,rate_of_penetration,drillbit_size,min_depth,max_depth,surface_weight_on_bit,surface_rpm
0,wellbore_chev_no_0_727,0,0,200,1,99.614650,-0.987049,0.387782,0.383551,0.579398,-0.233714
1,wellbore_chev_no_0_728,0,0,199,1,99.354021,-0.987049,0.388740,0.388778,0.493240,-0.233717
2,wellbore_chev_no_0_729,0,0,196,1,83.622907,-0.987049,0.393968,0.394006,0.964207,-0.231324
3,wellbore_chev_no_0_730,0,0,203,1,85.427731,-0.987049,0.399196,0.399234,-0.430323,-0.245178
4,wellbore_chev_no_0_731,0,0,203,1,100.639811,-0.987049,0.404423,0.404462,-0.328167,-0.247075


In [51]:
cat_cols = ["segment_id","rate_of_penetration","wellbore_chev_no_id","area_id","formation_id","bit_model_id"]


In [52]:
ID = df["segment_id"]
y = df.rate_of_penetration
df = df.drop(cat_cols,axis =1)

In [ ]:
#HYPEROPT
from hyperopt import hp, tpe, fmin

fixed_param = {
               "objective":"root_mean_squared_error",
              }

# This is the space that you want this function to explore
space_gbm = {
             'max_depth': hp.uniform('max_depth',5,70),
             'num_iterations': hp.uniform('num_iterations',50, 1000),
             'num_leaves': hp.uniform('num_leaves',10, 500),
             'min_data_in_leaf':hp.uniform('min_data_in_leaf',50, 500),
             'learning_rate': hp.uniform('learning_rate',0.001, 0.1)
            }
# THIS is the objective function that we want to minimize (note that we are multiplying the output by -1 as this function is always minimized never maximized)
def objective_gbm(params):
    cv = KFold(4, random_state=42,shuffle = True)
    li =[]
    # THis is CV in hyperopt
    for train, test in cv.split(df,y):
        X_train, X_test = df.values[train], df.values[test]
        y_train, y_test = y.values[train], y.values[test]
    
        params = {'num_iterations': int(params['num_iterations']),'max_depth': int(params['max_depth']),'num_leaves': int(params['num_leaves']),'min_data_in_leaf': int(params['min_data_in_leaf']),'learning_rate': float(params['learning_rate'])}
        all_params = fixed_param
        for i in params:
            all_params[i] = params[i]
        
        lgbm = gbm.LGBMRegressor(**all_params)
        lgbm = lgbm.fit(X_train,y_train)
        train_pred = lgbm.predict(X_train)
        test_pred = lgbm.predict(X_test)
        
        li.append(rmse(y_test,test_pred))
    output = (sum(li)/4)
    return output
# THis is the actual function that is run by this function
best_gbm = fmin(objective_gbm,
    space=space_gbm,
    algo=tpe.suggest,
    max_evals=50)


[LightGBM] [Warning] min_data_in_leaf is set=477, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=477
[LightGBM] [Warning] min_data_in_leaf is set=477, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=477
[LightGBM] [Warning] min_data_in_leaf is set=477, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=477
[LightGBM] [Warning] min_data_in_leaf is set=477, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=477
  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] min_data_in_leaf is set=128, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=128
  2%|▉                                                | 1/50 [00:00<00:11,  4.37trial/s, best loss: 1352.5845462442537]

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] min_data_in_leaf is set=128, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=128
  2%|▉                                                | 1/50 [00:00<00:11,  4.37trial/s, best loss: 1352.5845462442537]

...

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
 86%|█████████████████████████████████████████▎      | 43/50 [00:56<00:10,  1.51s/trial, best loss: 336.51318349800425]

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
 86%|█████████████████████████████████████████▎      | 43/50 [00:56<00:10,  1.51s/trial, best loss: 336.51318349800425]

C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



In [43]:
best_gbm = {'learning_rate': 0.06529246782268733,
 'max_depth': 39.80094910046667,
 'min_data_in_leaf': 97.59974683829947,
 'num_iterations': 955.3948383235338,
 'num_leaves': 459.8671565275702}

In [44]:
best_gbm["max_depth"] = int(best_gbm["max_depth"])
best_gbm["min_data_in_leaf"] = int(best_gbm["min_data_in_leaf"])
best_gbm["num_iterations"] = int(best_gbm["num_iterations"])
best_gbm["num_leaves"] = int(best_gbm["num_leaves"])


In [47]:
import lightgbm as gbm
parameters = best_gbm

lgbm = gbm.LGBMRegressor(**parameters)
lgbm.fit(df.values,y)

with open('LGBM.pickle', 'wb') as handle:
    pickle.dump(lgbm, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('LGBM.pickle', 'rb') as handle:
    lgbm_model = pickle.load(handle)
    for train_index, test_index in cv.split(df.values):
        X_train, X_test = df.values[train_index], df.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]

        train_pred = lgbm_model.predict(X_train)
        test_pred = lgbm_model.predict(X_test)
        print("Train predictions")
        print(rmse(y_train,train_pred))
        print("Test predictions")
        print(rmse(y_test,test_pred))
        print()


C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train predictions
62.89343996235424
Test predictions
64.81883448181041

Train predictions
62.16407746450416
Test predictions
67.00606137064341

Train predictions
62.666253836284746
Test predictions
65.50179768101749

Train predictions
65.77561835986484
Test predictions
56.17003307778181



In [25]:

cv = KFold(n_splits = 4)

for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index], df.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]

    lgbm = lgbm.fit(X_train,y_train)
    train_pred = lgbm.predict(X_train)
    test_pred = lgbm.predict(X_test)
    
    print("Train predictions")
    print(rmse(y_train,train_pred))
    print("Test predictions")
    print(rmse(y_test,test_pred))
    print()


C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=97, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=97
Train predictions
65.58113343717932
Test predictions
1674.9783352896418

[LightGBM] [Warning] min_data_in_leaf is set=97, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=97


C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train predictions
65.45059256851488
Test predictions
896.2718153176909

[LightGBM] [Warning] min_data_in_leaf is set=97, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=97


C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train predictions
67.88776846543819
Test predictions
784.9043794345362

[LightGBM] [Warning] min_data_in_leaf is set=97, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=97


C:\Users\TheDarkAce\Anaconda3\envs\deeplearning\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train predictions
68.33985182478615
Test predictions
920.049272034251



In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000,n_jobs = -1)

Train predictions
25.2494444551317
Test predictions
1391.839474014766

Train predictions
24.792514519087025
Test predictions
1066.9484534242451

Train predictions
25.056619098322468
Test predictions
802.8864884783625

Train predictions
26.510675889329214
Test predictions
1296.9851663844097



In [30]:
#HYPEROPT
from hyperopt import hp, tpe, fmin

# This is the space that you want this function to explore
space_gbm = {
             'max_depth': hp.uniform('max_depth',5,70),
             'n_estimators': hp.uniform('n_estimators',50, 1000),
             'min_samples_split': hp.uniform('min_samples_split',10, 500),
            }
# THIS is the objective function that we want to minimize (note that we are multiplying the output by -1 as this function is always minimized never maximized)
def objective_gbm(params):
    cv = KFold(4, random_state=42,shuffle = True)
    li =[]
    # THis is CV in hyperopt
    for train, test in cv.split(df,y):
        X_train, X_test = df.values[train], df.values[test]
        y_train, y_test = y.values[train], y.values[test]
    
        params = {'max_depth': int(params['max_depth']),'n_estimators': int(params['n_estimators']),'min_samples_split': int(params['min_samples_split'])}
        all_params = {}
        for i in params:
            all_params[i] = params[i]
        rf = RandomForestRegressor(**all_params)
        rf = rf.fit(X_train,y_train)
        train_pred = rf.predict(X_train)
        test_pred = rf.predict(X_test)
        
        li.append(rmse(test_pred,y_test))
    output = (sum(li)/4)
    return output
# THis is the actual function that is run by this function
best_gbm = fmin(objective_gbm,
    space=space_gbm,
    algo=tpe.suggest,
    max_evals=50)


100%|████████████████████████████████████████████████| 50/50 [15:52<00:00, 19.04s/trial, best loss: 222.52944546884814]


In [32]:
best_gbm

{'max_depth': 18.40734934755913,
 'min_samples_split': 11.088348377013457,
 'n_estimators': 739.1626726586705}

In [34]:
params = {'max_depth': 18,
 'min_samples_split': 11,
 'n_estimators': 739}

In [35]:
cv = KFold(n_splits = 4)

for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index], df.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    rf = RandomForestRegressor(**params)
    rf = rf.fit(X_train,y_train)
    train_pred = rf.predict(X_train)
    test_pred = rf.predict(X_test)
    print("Train predictions")
    print(rmse(y_train,train_pred))
    print("Test predictions")
    print(rmse(y_test,test_pred))
    print()

Train predictions
75.96827725565646
Test predictions
1350.4204352886886

Train predictions
82.32367283016865
Test predictions
1061.0361439387839

Train predictions
81.10949811511155
Test predictions
793.7667556544477

Train predictions
86.18040248949232
Test predictions
1390.990116417332



In [40]:
rf = RandomForestRegressor(**params)
rf.fit(df.values,y)
with open('randomforest.pickle', 'wb') as handle:
    pickle.dump(rf, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('randomforest.pickle', 'rb') as handle:
    rf_model = pickle.load(handle)
    for train_index, test_index in cv.split(df.values):
        X_train, X_test = df.values[train_index], df.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]

        train_pred = rf_model.predict(X_train)
        test_pred = rf_model.predict(X_test)
        print("Train predictions")
        print(rmse(y_train,train_pred))
        print("Test predictions")
        print(rmse(y_test,test_pred))
        print()

Train predictions
79.86548870185706
Test predictions
84.24547985310905

Train predictions
80.51048495756399
Test predictions
82.31125214351708

Train predictions
81.1709548650675
Test predictions
80.33012567211105

Train predictions
82.29600582667926
Test predictions
76.95364450987122



In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [13]:

for i in df.columns:
    if df[i].dtype == "int64":
        df[i] = df[i].astype("object")
    
    print(df[i].dtype)
df_nn = pd.get_dummies(df)

object
object
object
object
float64
float64
float64
float64
float64


In [14]:
df_nn.shape

(6778, 305)

In [15]:

inputs = keras.Input(shape = 305)
x = layers.Dense(1000,activation = "relu")(inputs)
x = layers.Dense(800,activation = "relu")(x)
x = layers.Dense(600,activation = "relu")(x)
x = layers.Dense(400,activation = "relu")(x)
x = layers.Dense(200,activation = "relu")(x)
x = layers.Dense(100,activation = "relu")(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs,outputs)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["MeanSquaredError"],
)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 305)]             0         
_________________________________________________________________
dense (Dense)                (None, 1000)              306000    
_________________________________________________________________
dense_1 (Dense)              (None, 800)               800800    
_________________________________________________________________
dense_2 (Dense)              (None, 600)               480600    
_________________________________________________________________
dense_3 (Dense)              (None, 400)               240400    
_________________________________________________________________
dense_4 (Dense)              (None, 200)               80200     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               20100 

In [18]:
cv = KFold(n_splits = 4)

for train_index, test_index in cv.split(df_nn.values):
    X_train, X_test = df_nn.values[train_index], df_nn.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    history = model.fit(X_train, y_train, batch_size=64, epochs=100,verbose=1)
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print(train_pred.shape)
    print("Train predictions")
    print(rmse(y_train,train_pred))
    print("Test predictions")
    print(rmse(y_test,test_pred))
    print()


Epoch 1/100
80/80 [==============================] - 3s 4ms/step - loss: 1876.4877 - mean_squared_error: 1876.4877
Epoch 2/100
80/80 [==============================] - 0s 2ms/step - loss: 367.8021 - mean_squared_error: 367.8021
Epoch 3/100
80/80 [==============================] - 0s 2ms/step - loss: 268.0180 - mean_squared_error: 268.0180
Epoch 4/100
80/80 [==============================] - 0s 2ms/step - loss: 258.0678 - mean_squared_error: 258.0678
Epoch 5/100
80/80 [==============================] - 0s 2ms/step - loss: 237.2225 - mean_squared_error: 237.2225
Epoch 6/100
80/80 [==============================] - 0s 2ms/step - loss: 232.2918 - mean_squared_error: 232.2918
Epoch 7/100
80/80 [==============================] - 0s 2ms/step - loss: 221.5451 - mean_squared_error: 221.5451
Epoch 8/100
80/80 [==============================] - 0s 2ms/step - loss: 231.6370 - mean_squared_error: 231.6370
Epoch 9/100
80/80 [==============================] - 0s 2ms/step - loss: 218.1162 - mean_squar

In [19]:
df.head()

,wellbore_chev_no_id,area_id,formation_id,bit_model_id,drillbit_size,min_depth,max_depth,surface_weight_on_bit,surface_rpm
0,0,0,200,1,-0.987049,0.387782,0.383551,0.579398,-0.233714
1,0,0,199,1,-0.987049,0.388740,0.388778,0.493240,-0.233717
2,0,0,196,1,-0.987049,0.393968,0.394006,0.964207,-0.231324
3,0,0,203,1,-0.987049,0.399196,0.399234,-0.430323,-0.245178
4,0,0,203,1,-0.987049,0.404423,0.404462,-0.328167,-0.247075


In [21]:
from catboost import CatBoostRegressor


cv = KFold(n_splits = 4)
cat_features = [0,1,2,3]

for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index], df.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    cat = CatBoostRegressor(
        iterations=1000, 
        learning_rate=0.001, 
        loss_function='RMSE'
    )
    cat.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_test, y_test)
    )
    train_pred = cat.predict(X_train)
    test_pred = cat.predict(X_test)
    
    print(train_pred.shape)
    print("Train predictions")
    print(rmse(y_train,train_pred))
    print("Test predictions")
    print(rmse(y_test,test_pred))
    print()
    

0:	learn: 44.8859646	test: 58.1540952	best: 58.1540952 (0)	total: 187ms	remaining: 3m 7s
1:	learn: 44.8557971	test: 58.1130280	best: 58.1130280 (1)	total: 225ms	remaining: 1m 52s
2:	learn: 44.8272445	test: 58.0801689	best: 58.0801689 (2)	total: 258ms	remaining: 1m 25s
3:	learn: 44.8007867	test: 58.0543055	best: 58.0543055 (3)	total: 290ms	remaining: 1m 12s
4:	learn: 44.7735200	test: 58.0186307	best: 58.0186307 (4)	total: 323ms	remaining: 1m 4s
5:	learn: 44.7455351	test: 57.9856026	best: 57.9856026 (5)	total: 366ms	remaining: 1m
6:	learn: 44.7183564	test: 57.9579732	best: 57.9579732 (6)	total: 414ms	remaining: 58.7s
7:	learn: 44.6904066	test: 57.9270172	best: 57.9270172 (7)	total: 458ms	remaining: 56.8s
8:	learn: 44.6602696	test: 57.8894251	best: 57.8894251 (8)	total: 486ms	remaining: 53.5s
9:	learn: 44.6328513	test: 57.8592630	best: 57.8592630 (9)	total: 514ms	remaining: 50.9s
10:	learn: 44.6046798	test: 57.8199593	best: 57.8199593 (10)	total: 542ms	remaining: 48.7s
11:	learn: 44.57580

In [22]:
rmse([2,2],[0,0])

4.0

In [ ]:
## Autoencoders

In [23]:
inputs = keras.Input(shape = 305)
x = layers.Dense(1000,activation = "relu")(inputs)
x = layers.Dense(800,activation = "relu")(x)
x = layers.Dense(600,activation = "relu")(x)
x = layers.Dense(400,activation = "relu")(x)
x = layers.Dense(1,activation = "relu")(x)
x = layers.Dense(400,activation = "relu")(x)
x = layers.Dense(600,activation = "relu")(x)
x = layers.Dense(800,activation = "relu")(x)
x = layers.Dense(1000,activation = "relu")(x)
outputs = layers.Dense(305)(x)
model = keras.Model(inputs,outputs)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["MeanSquaredError"],
)

In [24]:
cv = KFold(n_splits = 4)

for train_index, test_index in cv.split(df_nn.values):
    X_train, X_test = df_nn.values[train_index], df_nn.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    history = model.fit(X_train, X_train, batch_size=64, epochs=100,verbose=1)
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print(X_train.shape)
    print(train_pred.shape)
    print("Train predictions")
    print(rmse(X_train,train_pred))
    print("Test predictions")
    print(rmse(X_test,test_pred))
    print()

Epoch 1/100
80/80 [==============================] - 1s 5ms/step - loss: 0.0173 - mean_squared_error: 0.0173
Epoch 2/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0136 - mean_squared_error: 0.0136
Epoch 3/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0118 - mean_squared_error: 0.0118
Epoch 4/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0109 - mean_squared_error: 0.0109
Epoch 5/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0102 - mean_squared_error: 0.0102
Epoch 6/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 7/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0093 - mean_squared_error: 0.0093
Epoch 8/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0091 - mean_squared_error: 0.0091
Epoch 9/100
80/80 [==============================] - 0s 3ms/step - loss: 0.0088 - mean_squared_error: 0.0088
Epoch 10/100
80/80 

In [212]:
model.layers[0]

In [27]:
model2 = keras.Model(inputs=model.input, outputs=model.layers[-6].output)
print(model2)

In [28]:
model2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 305)]             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1000)              306000    
_________________________________________________________________
dense_8 (Dense)              (None, 800)               800800    
_________________________________________________________________
dense_9 (Dense)              (None, 600)               480600    
_________________________________________________________________
dense_10 (Dense)             (None, 400)               240400    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 401       
Total params: 1,828,201
Trainable params: 1,828,201
Non-trainable params: 0
_________________________________________________

In [227]:
for layer in model2.layers[:]:
    layer.trainable = False

In [229]:
inputs2 = keras.Input(shape = 305)
x = model2()(inputs2)
x = layers.Dense(100,activation = "relu")(x)
outputs2 = layers.Dense(1)(x)
model3 = keras.Model(inputs2,outputs2)


ValueError: The first argument to `Layer.call` must always be passed.

In [225]:
model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 305)]             0         
_________________________________________________________________
dense_79 (Dense)             (None, 1000)              306000    
_________________________________________________________________
dense_80 (Dense)             (None, 800)               800800    
_________________________________________________________________
dense_81 (Dense)             (None, 600)               480600    
_________________________________________________________________
dense_82 (Dense)             (None, 400)               240400    
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 401       
_________________________________________________________________
dense_84 (Dense)             (None, 400)               800